# Objectifs

+ Obtenir et distribuer des paires bound (protein + ssDNA) / unbound (proteine seules)

In [1]:
update_biounit = True
clusterNumber = 100
#clusterNumber = 30

In [2]:
from utils.utils_rcsb import *
from utils.utils_bash import *
from utils.utils_benchmark import *
from tqdm.notebook import tqdm
import re

from rcsbsearch import Attr

In [3]:
from vmd import molecule, atomsel
from pdbfixer import pdbfixer
from simtk.openmm.app import PDBFile

from glob import glob
import subprocess
from itertools import groupby, count, chain, combinations, permutations
from collections.abc import Iterable
import os, json, sys
import requests, gzip
import numpy as np
import networkx as nx

In [4]:
from os.path import getmtime
import time

In [5]:
import subprocess
from multiprocessing.pool import ThreadPool

# Find DNA/protein and protein only chains

Get PDBid of bound and unbound proteins

In [6]:
prot_q = open("./pdb_query/protein.txt", "r").read().split("\n")
dna_q = open("./pdb_query/dna.txt", "r").read().split("\n")
rna_q = open("./pdb_query/rna.txt", "r").read().split("\n")
hyb_q = open("./pdb_query/hybrid.txt", "r").read().split("\n")

In [7]:
protein_with_dna = set(prot_q) & set(dna_q) - set(rna_q) - set(hyb_q)
protein_without_dna = set(prot_q) - set(dna_q) - set(rna_q) - set(hyb_q)

In [8]:
len(protein_with_dna), len(protein_without_dna)

(3030, 121459)

# Download bound PDBs

In [9]:
def download_file(url, target):
    r = requests.get(url, stream=True)
    if r.status_code == 404:
        return False
    # Total size in bytes.
    total_size = int(r.headers.get('content-length', 0))
    block_size = 1024 #1 Kibibyte
    with open(target, 'wb') as f:
        for data in r.iter_content(block_size):
            f.write(data)
    return True

def download_file_stream(url, target, pb = tqdm):
    # Streaming, so we can iterate over the response.
    r = requests.get(url, stream=True)
    # Total size in bytes.
    total_size = int(r.headers.get('content-length', 0))
    block_size = 1024 #1 Kibibyte
    t=tqdm(total=total_size, unit='iB', unit_scale=True)
    with open(target, 'wb') as f:
        for data in r.iter_content(block_size):
            t.update(len(data))
            f.write(data)
    t.close()

## bio assemblies

In [10]:
# Only download them if they are older than 1 hour
ct = time.time()
url = 'http://files.rcsb.org/pub/pdb/data/biounit/coordinates/all'
target = "all.txt"
try:
    ft = getmtime(target)
    if ct-ft > 3600:
        download_file_stream(url, target)
except:
    download_file_stream(url, target)

In [11]:
all_biounit = open("all.txt", "r").read()
all_biounit_files = uniq(re.findall("(.{4}.pdb[0-9]+\.gz)", open("all.txt", "r").read()))

In [12]:
protein_with_dna = [i.lower() for i in protein_with_dna]
biounit_files_withDNA = uniq([biounit_f for biounit_f in tqdm(all_biounit_files) if biounit_f[:4] in protein_with_dna])

  0%|          | 0/258269 [00:00<?, ?it/s]

In [13]:
def dl_bio_unit(f):
    f = f.lower()
    ####
    if os.path.exists(f"pdb_bio/{f}"):
        os.remove(f"pdb_bio/{f}")
    if os.path.exists(f"pdb_bio/{f[:-3]}"):
        return
    ####
    url = f"https://files.rcsb.org/download/{f}"
    download_file(url, f"pdb_bio/{f}")
    
    try:
        with gzip.open(f"pdb_bio/{f}", "rb") as my_f:
            data = my_f.read()
            open(f"pdb_bio/{f[:-3]}", "wb").write(data)

        os.remove(f"pdb_bio/{f}")
    except:
        pass
    
# Run 50 multiple threads. Each call will take the next element in urls list
results = ThreadPool(50).imap_unordered(dl_bio_unit, biounit_files_withDNA)
for r in tqdm(results, total = len(biounit_files_withDNA)):
    pass

  0%|          | 0/3910 [00:00<?, ?it/s]

In [14]:
# Download biounit if not existing
for f in tqdm(biounit_files_withDNA, total = len(biounit_files_withDNA)):
    f = f.lower()
    ####
    if os.path.exists(f"pdb_bio/{f}"):
        os.remove(f"pdb_bio/{f}")
    if os.path.exists(f"pdb_bio/{f[:-3]}"):
        continue
    ####
    url = f"https://files.rcsb.org/download/{f}"
    download_file(url, f"pdb_bio/{f}")
    
    with gzip.open(f"pdb_bio/{f}", "rb") as my_f:
        data = my_f.read()
        open(f"pdb_bio/{f[:-3]}", "wb").write(data)
    
    os.remove(f"pdb_bio/{f}")

  0%|          | 0/3910 [00:00<?, ?it/s]

## Main structures

In [15]:
failed_dl = []
def dl_str(f):
    f = f.lower()
    ####
    if os.path.exists(f"pdb_str/{f}.pdb"):
        return
    ####
    url = f"https://files.rcsb.org/download/{f}.pdb"
    success = download_file(url, f"pdb_str/{f}.pdb")
    if success == False:
#         print(f)
        failed_dl.append(f)
    
# Run 5 multiple threads. Each call will take the next element in urls list
results = ThreadPool(50).imap_unordered(dl_str, protein_with_dna)
for r in tqdm(results, total = len(protein_with_dna)):
    pass

  0%|          | 0/3030 [00:00<?, ?it/s]

In [16]:
failed_dl

['6uua',
 '6vu3',
 '6vmi',
 '6uty',
 '6vlz',
 '6uub',
 '6vz3',
 '7ena',
 '7enc',
 '7egc',
 '6vz2',
 '7omz',
 '7on0',
 '7b23',
 '7cr8',
 '6vyq',
 '6utx',
 '6vyt',
 '6vyr',
 '6vyu',
 '7egb',
 '6vys',
 '7b24',
 '7b20',
 '6vz7',
 '7omy',
 '6vz5',
 '7b1y']

In [18]:
for i,r in df_bound.iterrows():
    if "6WE" in r.boundId:
        print(i, r.boundId)

NameError: name 'df_bound' is not defined

# Analyse for ssDNA

The idea is to analyse the main structure (first frame) or the biological assemblies (all frames merged) to identify dsDNA, and therefor ssDNA

## Functions

In [18]:
def get_consecutive_elements(data):
    c = count()
    val = list(list(g) for _, g in groupby(data, lambda x: x-next(c)))
    return val

## find_pair analysis (find dsDNA)

In [19]:
def analyse_findPair(s):
    try:
        ### Analyse main structure
        str_pdb = glob(f"pdb_str/{s}.pdb")[0]
        fatcat_pdb = str_pdb.replace("pdb_str", "results_findPair")\
                                .replace("pdb", "fp")
        if not os.path.exists(fatcat_pdb):
            fp, fp_err = find_pair(str_pdb)
            with open(fatcat_pdb, "w") as f:
                f.write(fp.replace("\\n", "\n"))

        ### Analyse biological assembly
        str_bio = sorted(glob(f"pdb_bio/{s}.pdb[0-9]"))
        for b in str_bio:
            fatcat_b = b.replace("pdb_bio", "results_findPair")\
                        .replace("pdb", "fp")
            if not os.path.exists(fatcat_b):
                # Only keep ATOM records
                nb = open(b, "r").read().split("\n")
                nb = [line for line in nb if line.startswith("ATOM")]
                nb = "\n".join(nb)
                open(f"tmp_findpair.{s}.pdb", "w").write(nb)
                # Find pair
                fp, fp_err = find_pair(f"tmp_findpair.{s}.pdb")
                with open(fatcat_b, "w") as f:
                    f.write(fp.replace("\\n", "\n"))
        

    except Exception as e:
        print(s, e)
        return

    try:
        os.remove(f"tmp_findpair.{s}.pdb")
    except:
        pass
    
do_fp = False
try:
    ct = time.time()
    ft = getmtime("last_run.txt")
    if ct-ft > 3600*12:
        do_fp = True
except:
    open("last_run.txt", "w").write("")
    do_fp = True

for f in tqdm(protein_with_dna):
    analyse_findPair(f)

  0%|          | 0/3002 [00:00<?, ?it/s]

## List DNA close to protein

In [20]:
do_run = False

ct = time.time()
ft = getmtime("last_run.txt")
if ct-ft > 3600*12:
    do_run = True
####    ####    ####    ####

for s in tqdm(protein_with_dna):
    if not do_run:
        break
    try:
        ### Analyse main structure
        str_pdb = glob(f"pdb_str/{s}.pdb")[0]
        lsDNA_pdb = str_pdb.replace("pdb_str", "results_listDNA")\
                                .replace("pdb", "list")
        if not os.path.exists(lsDNA_pdb):
            mol = molecule.load("pdb", str_pdb)
             ##!! Sidechain/base selection
            sel = atomsel("(within 5 of protein) and nucleic and altloc 'A' '' ' ' and not name OP1 C5' OP2 C3' O5' C1' O4' O3' C4' C2' P", mol)
            with open(lsDNA_pdb, "w") as f:
                for i,j,k in sorted(uniq(list(zip(sel.chain, sel.resid, [r[1:] if r[0] == "D" else r for r in sel.resname]))), key = lambda x:(x[0], x[1])):
                    f.write(str(i)+"\t"+str(j)+"\t"+str(k)+"\n")
            molecule.delete(mol)        

    except Exception as e:
        print(s, e)
        continue

  0%|          | 0/3002 [00:00<?, ?it/s]

## Analyse main str and bio-ass.

In [21]:
ssDNA_dict = {}
min_ssDNA_length = 4

for s in tqdm(protein_with_dna):
#     ssDNA[s] = []
    try:
        # find pair analysis
        fp_main = open(f"results_findPair/{s}.fp").read()

        # Replace "same chain"interactions
        fp_main = [line for line in fp_main.split("\n") if len(re.findall(">(.):.+(\*\*).+:(.)<", line)) == 0]
        fp_main = "\n".join(fp_main)
#         "\n".join([line for line in fp_main.split("\n") if ((re.findall(">(.):", line) != re.findall(":(.)<", line)) and (len(re.findall(">(.):.+(\*\*).+:(.)<", fp_main)) == 0) )])
        
        fp_bio  = [open(sb).read() for sb in sorted(glob(f"results_findPair/{s}.fp[0-9]"))]
        lst_bp_main =  re.findall(">(.):\.*([\-0-9]+).:", fp_main)+ [(i[1], i[0]) for i in re.findall(":\.*([\-0-9]+).:(.)<", fp_main)]
        lst_bp_bio  = [re.findall(">(.):\.*([\-0-9]+).:", fb)      + [(i[1], i[0]) for i in re.findall(":\.*([\-0-9]+).:(.)<", fb)] for fb in fp_bio]
        #s If they are double stranded in at least one of the structure (main or bio-assembly)
        lst_bp = uniq(lst_bp_main + list(chain.from_iterable(lst_bp_bio)))
        
        # All DNA residues analysis
        ld = open(f"results_listDNA/{s}.list").read()
        ld = [(i.split("\t")[0], i.split("\t")[1]) for i in ld.split("\n") if i != ""]
        
        # DNA residues not in double strand => ssDNA
        ssDNA = set(ld) - set(lst_bp)
        
        # List ssDNA chains
        ssDNA_ch = {i[0]:sorted([int(j[1]) for j in ssDNA if j[0] == i[0]]) for i in ssDNA}
        # Check for length
        ssDNA_cons = {k:get_consecutive_elements(v) for k,v in ssDNA_ch.items()}
        ssDNA_cons = {k:[i for i in v if len(i) >= min_ssDNA_length] for k,v in ssDNA_cons.items()}
        ssDNA_cons = {k:v for k,v in ssDNA_cons.items() if len(v) > 0}
        
        ## ## ##
        if len(ssDNA_cons) > 0:
            ssDNA_dict[s] = ssDNA_cons
        
    except Exception as e:
        print(s, e)
        print(e.with_traceback())
        continue
    if True:
        pass
#     break

  0%|          | 0/3002 [00:00<?, ?it/s]

## Find protein interacting chains

In [22]:
protein_to_DNA_interactions = pd.DataFrame(columns = ["structureId", "chainId", "int_ssDNA_chain", "ssDNA_int"])
row_count = 0

for s, inter in tqdm(ssDNA_dict.items()):
    
    sel_list = [["protein and within 5 of (chain '"+ss_chain +"' and resid "+ ' '.join(map(lambda x: "'"+str(x)+"'", sub_ss_int)) +")", ss_chain, sub_ss_int]
                        for ss_chain, ss_int in inter.items()
                        for sub_ss_int in ss_int]
    mol = molecule.load("pdb", f"pdb_str/{s}.pdb")
    
    prot_int = []
    # Find protein chains close to the ssDNA
    for sel_txt, dna_ch, dna_inter in sel_list:
        try:
            sel = atomsel(sel_txt, mol)
        except Exception as e:
            print(s, e)
        prot_int.append([dna_ch, dna_inter, uniq(sel.chain)])
    
    # For each previously found protein chain, find consecutive interacting NA chains longer than 4 residues
    for dna_ch, dna_int, prot_ch in prot_int:
        for ch in prot_ch:
            if ch == dna_ch:
                print(s, dna_ch, ch)
            sel_txt = "chain '" + dna_ch + "' and resid " + " ".join(map(lambda x: "'"+str(x)+"'", dna_int)) + " and (within 5 of chain '" + ch + "')"
            sel = atomsel(sel_txt)
            
            cons_dna = get_consecutive_elements(sorted(uniq(sel.resid)))
            cons_dna = [i for i in cons_dna if len(i) >= min_ssDNA_length]
            if len(cons_dna) == 1:
                pass
            elif len(cons_dna) == 0:
                continue
            else:
                new_cons = [cons_dna[0]]
                for l0, l1 in zip(cons_dna[:-1], cons_dna[1:]):
                    # If a 1-gap is found, fill it
                    if l1[0]-l0[-1] == 2:
                        new_cons[-1] = new_cons[-1]+[l1[0]-1]+l1
                    else:
                        new_cons.append(l1)
                cons_dna = new_cons
            
            # Update Dataframe
            protein_to_DNA_interactions.loc[row_count]=[s.upper(), ch, dna_ch, str(cons_dna)]
            row_count += 1
    
    molecule.delete(mol)
protein_to_DNA_interactions.to_csv("./csv_tables/df_protein_dna_interaction.csv", float_format='%.1f')

  0%|          | 0/317 [00:00<?, ?it/s]

In [23]:
bound_pdb_chains = uniq([i+"."+j for i,j in zip(protein_to_DNA_interactions.structureId, protein_to_DNA_interactions.chainId)])
len(bound_pdb_chains)

565

# Complete ssDNA-protein interactions table

In [24]:
protein_to_DNA_interactions = pd.read_csv("./csv_tables/df_protein_dna_interaction.csv", index_col = 0)

In [25]:
q="""{entries(entry_ids:[\""""+"\",\"".join(uniq(protein_to_DNA_interactions.structureId))+"""\"]){rcsb_id,struct_keywords{pdbx_keywords}exptl{method}polymer_entities{entity_poly{type}rcsb_cluster_membership{cluster_id,identity}rcsb_polymer_entity_container_identifiers {auth_asym_ids}}}}"""
response = urllib.request.urlopen("https://data.rcsb.org/graphql?query=" + urllib.parse.quote(q.replace("\n", "").replace(" ", "")))
r = response.read().decode("utf-8")
j = json.loads(r)
j = j["data"]["entries"]

In [26]:
data = []
for i in j:    
    exp_met = i['exptl'][0]["method"]
    desc = i['struct_keywords']["pdbx_keywords"]
    try:
        for k in i['polymer_entities']:
            entity_type = k["entity_poly"]["type"]
            if "polypeptide" in entity_type:
#                 print("k", k)
                for l in k["rcsb_cluster_membership"]:
                    if l["identity"] == 100:
                        for c in k["rcsb_polymer_entity_container_identifiers"]["auth_asym_ids"]:
                            data.append([i["rcsb_id"], c, l['cluster_id'], entity_type, exp_met, desc])
    except:
        continue
prot_to_dna_info = pd.DataFrame(data, columns=["structureId", "chainId", "clusterNumber100", "entity_type", "experimentalTechnique", "classification"])
prot_to_dna_info.dropna(inplace=True)

In [27]:
df_prot_to_dna_info = protein_to_DNA_interactions.merge(prot_to_dna_info, left_on = ["structureId", "chainId"], right_on = ["structureId", "chainId"])
only_interactions = df_prot_to_dna_info[["structureId", "chainId", "int_ssDNA_chain", "ssDNA_int"]]
df_with_ssdna = df_prot_to_dna_info[["structureId", "chainId", "clusterNumber100", "experimentalTechnique", "classification"]]
df_with_ssdna = df_with_ssdna.drop_duplicates()

In [28]:
only_interactions.to_csv("csv_tables/ssDNA_interacting.csv", float_format='%.1f')

## Save ssDNA + protein chains

In [29]:
only_interactions

,structureId,chainId,int_ssDNA_chain,ssDNA_int
0,4A8Q,C,H,"[[3, 4, 5, 6]]"
1,6F2S,F,P,"[[3, 4, 5, 6, 7]]"
2,6F2S,E,O,"[[3, 4, 5, 6, 7]]"
3,6F2S,H,S,"[[3, 4, 5, 6]]"
4,6F2S,I,S,"[[3, 4, 5, 6]]"
...,...,...,...,...
628,4TU9,B,E,"[[3, 4, 5, 6]]"
629,4TU9,B,P,"[[1, 2, 3, 4, 5, 6]]"
630,4TU9,A,P,"[[3, 4, 5, 6]]"
631,2ES2,A,B,"[[2, 3, 4, 5]]"


In [30]:
def get_line_description(line):
    return line[12:16] + " " + line[17:20] + " " + line[22:26]

for mol in molecule.listall():
    molecule.delete(mol)
    
for i, (s, pc, dc, _) in tqdm(only_interactions.iterrows(), total = len(only_interactions)):
    s = s.lower()
    chain_name = f"pdb_ssDNAProtChains/{s}_{pc}_{dc}.pdb"
    if os.path.exists(chain_name):
        continue
    
    mol = molecule.load("pdb", f"./pdb_str/{s}.pdb")
    sel = atomsel(f"noh and chain {pc} {dc}")
    
    sel.write("pdb", chain_name)
    
    molecule.delete(mol)
    
        #Delete duplicate lines (~Altloc)
    pdb_content = open(chain_name, "r").read().split("\n")
    for line in range(len(pdb_content)-1, 0, -1):
        if get_line_description(pdb_content[line]) == get_line_description(pdb_content[line-1]):
            del(pdb_content[line])
    open(chain_name, "w").write("\n".join(pdb_content))
    
    #Add missing atoms, remove water, ...
#     try:
#         fix = pdbfixer.PDBFixer(filename=chain_name)
#         fix.findMissingResidues()
#         fix.findMissingAtoms()
#         fix.addMissingAtoms()
#         fix.removeHeterogens(keepWater=False)
#         fix.topology.createStandardBonds()
#         PDBFile.writeFile(fix.topology, fix.positions, open(chain_name, 'w'), keepIds = True)
#     except:
#         print("PDB fix err.", chain_name)
#         continue

  0%|          | 0/633 [00:00<?, ?it/s]

# Make tables

## Merge Bound structures by cluster

In [31]:
df_bound = pd.DataFrame(columns=[f"clusterNumber{clusterNumber}",'boundId','bound_TechExp','bound_class'])
clusters_bound = map(np.int64, sorted(map(int, uniq(df_with_ssdna[f"clusterNumber{clusterNumber}"]))))

for cluster in clusters_bound:
    pd_bound_cluster = df_with_ssdna[df_with_ssdna[f"clusterNumber{clusterNumber}"] == cluster]
    bound_id_ch = [i+"_"+j for i, j in zip(pd_bound_cluster.structureId, pd_bound_cluster.chainId)]
    bound_techExp = uniq([i for i in pd_bound_cluster.experimentalTechnique])
    bound_class = uniq([i.lower() for i in pd_bound_cluster.classification])
    
    df_bound = df_bound.append({f"clusterNumber{clusterNumber}":cluster,
                                 'boundId':" ".join(bound_id_ch),
                                 'bound_TechExp':", ".join(bound_techExp),
                                 'bound_class':", ".join(bound_class)
                               }, 
                               ignore_index=True)

In [32]:
df_bound

,clusterNumber100,boundId,bound_TechExp,bound_class
0,118,1RCN_E,X-RAY DIFFRACTION,hydrolase/dna
1,144,6N4C_C 6GH5_C 6XL5_C 6PST_I 3IYD_C 6XH7_C 7C17...,ELECTRON MICROSCOPY,"transcription, transferase/dna, transcription,..."
2,153,6GH5_D 6XL5_D 3IYD_D 6Z9T_Y 6GFW_D,ELECTRON MICROSCOPY,"transcription, transferase/dna, transcription,..."
3,161,7NKY_B,ELECTRON MICROSCOPY,transcription
4,207,2AGP_A,X-RAY DIFFRACTION,transferase/dna
...,...,...,...,...
204,95414,6RWL_F,ELECTRON MICROSCOPY,recombination
205,98924,5VI5_C,X-RAY DIFFRACTION,transcription
206,100737,6DG0_A 6DG0_B,X-RAY DIFFRACTION,rna binding protein/dna
207,101742,6BUX_A,X-RAY DIFFRACTION,hydrolase/dna


## Add unbound structures by cluster

In [33]:
def get_cluster_by_id(cluster_id:int):
    """
    cluster_id : ID of the wanted cluster
    return : 
        "structure.chain" list in the cluster
    """
    q1 = Attr('rcsb_cluster_membership.cluster_id').equals(int(cluster_id))
    q2 = Attr('rcsb_cluster_membership.identity').equals(100)
    q = q1 & q2
    r = list(set(q("polymer_instance")))
    return r

protein_without_dna = set(protein_without_dna)
df_bound_unbound = df_bound.copy()
df_bound_unbound["unboundId"] = ["" for i in range(len(df_bound_unbound))]

# For each cluster in the dataframe...
for i, r in tqdm(df_bound_unbound.iterrows(), total = len(df_bound_unbound)):
    (cl, b, b_ET, b_class, ub) = r
    b_for_query = b.split(" ")[0].replace("_", ".")
    
    cl_pdb_ch = get_cluster_by_id(cl)
    ub_pdb_ch = [s.replace(".", "_") for s in cl_pdb_ch if s[:4] in protein_without_dna]
    
    df_bound_unbound.iloc[i]["unboundId"] = " ".join(ub_pdb_ch)

df_bound_unbound.replace('', np.nan, inplace=True)
df_bound_unbound.dropna(inplace=True)

  0%|          | 0/209 [00:00<?, ?it/s]

In [34]:
df_bound_unbound.shape

(105, 5)

## Convert unbound chains into author's chains
There is some discrepancy between the two id (Auth. 1RCN_E = 1RCN.B)

In [35]:
ub_q = "\""+"\",\"".join(df_bound_unbound["unboundId"]).replace("_", ".").replace(" ", "\",\"") + "\""
q="""{polymer_entity_instances(instance_ids: [""" + ub_q+ """]) {rcsb_id,rcsb_polymer_entity_instance_container_identifiers {auth_asym_id}}}"""
response = urllib.request.urlopen("https://data.rcsb.org/graphql?query=" + urllib.parse.quote(q.replace("\n", "").replace(" ", "")))
r = response.read().decode("utf-8")
j = json.loads(r)
j = j["data"]["polymer_entity_instances"]

In [36]:
id_to_authId = {}

for i in j:
    rcsb_id = i["rcsb_id"].split(".")[0]
    rcsb_ch = i["rcsb_id"].split(".")[-1]
    auth_id = i["rcsb_polymer_entity_instance_container_identifiers"]["auth_asym_id"]
    
    if rcsb_ch != auth_id:
        id_to_authId[rcsb_id+"_"+rcsb_ch] = rcsb_id+"."+auth_id

In [37]:
for k,v in tqdm(id_to_authId.items()):
    df_bound_unbound["unboundId"] = df_bound_unbound["unboundId"].str.replace(k,v)

df_bound_unbound["unboundId"] = df_bound_unbound["unboundId"].str.replace(".","_")

  0%|          | 0/249 [00:00<?, ?it/s]

/home/dominique/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.


In [38]:
df_bound_unbound[df_bound_unbound.clusterNumber100 == 5616]

,clusterNumber100,boundId,bound_TechExp,bound_class,unboundId


In [39]:
df_bound_unbound.shape

(105, 5)

<!-- ## Bound/unbound table : merge bound and unbound lists -->

## Add unbound experimental info

In [40]:
all_unbound = " ".join(df_bound_unbound.unboundId).replace("_", ".").split(" ")
all_unbound = uniq([i[:4] for i in all_unbound])
ub_q = "\""+"\",\"".join(all_unbound) + "\""

In [41]:
q = """{entries(entry_ids: [""" + ub_q+ """]) {rcsb_id,struct_keywords {pdbx_keywords}exptl {method}}}
"""
response = urllib.request.urlopen("https://data.rcsb.org/graphql?query=" + urllib.parse.quote(q.replace("\n", "").replace(" ", "")))
r = response.read().decode("utf-8")
j = json.loads(r)
j = j["data"]["entries"]

In [42]:
meth_dict = {}
desc_dict = {}

for entry in j:
    pdb_id = entry["rcsb_id"]
    desc = entry["struct_keywords"]["pdbx_keywords"]
    meth = entry["exptl"][0]["method"]
    meth_dict[pdb_id] = meth
    desc_dict[pdb_id] = desc

In [43]:
df_bound_unbound["unbound_TechExp"] = ["" for i in range(len(df_bound_unbound))]
df_bound_unbound["unbound_class"] = ["" for i in range(len(df_bound_unbound))]

unbound_TechExp = []
unbound_class = []

for i, r in tqdm(df_bound_unbound.iterrows(), total = len(df_bound_unbound)):
    unbound_pdb = uniq([v[:4] for v in r.unboundId.split(" ")])
    
    meth = uniq([meth_dict[pdb].upper() for pdb in unbound_pdb])
    desc = uniq([desc_dict[pdb].lower() for pdb in unbound_pdb])
    
    try:
        unbound_TechExp.append(" ".join(meth))
        unbound_class.append(" ".join(desc))
    except Exception as e:
        print(e, i)
df_bound_unbound["unbound_TechExp"] = unbound_TechExp
df_bound_unbound["unbound_class"] = unbound_class

  0%|          | 0/105 [00:00<?, ?it/s]

In [44]:
# Merge functional classifications
b_class = df_bound_unbound.bound_class
ub_class = df_bound_unbound.unbound_class
merge_class = [i+","+j for i, j in zip(b_class, ub_class)]
merge_class = [i.split(",") for i in merge_class]
merge_class = [",".join(uniq(i)) for i in merge_class]
df_bound_unbound["classification"] = merge_class
# clean
df_bound_unbound = df_bound_unbound[["clusterNumber100","boundId","unboundId","bound_TechExp","unbound_TechExp","classification"]]
# df_bound_unbound = df_bound_unbound.rename(columns={"unboundId_x":"unboundId"})

In [45]:
df_bound_unbound

,clusterNumber100,boundId,unboundId,bound_TechExp,unbound_TechExp,classification
0,118,1RCN_E,4OT4_B 1A2W_A 1RNU_A 3LXO_A 1RNO_A 1WBU_B 1RNW...,X-RAY DIFFRACTION,X-RAY DIFFRACTION NEUTRON DIFFRACTION,"hydrolase/dna,rna) hydrolase(phosphoric dieste..."
1,144,6N4C_C 6GH5_C 6XL5_C 6PST_I 3IYD_C 6XH7_C 7C17...,4KN7_C 6PSS_I 4JKR_I 5VT0_I 4LK1_I 5W1S_I 4XSX...,ELECTRON MICROSCOPY,X-RAY DIFFRACTION ELECTRON MICROSCOPY,"transcription/dna, transcription, transferase..."
2,153,6GH5_D 6XL5_D 3IYD_D 6Z9T_Y 6GFW_D,4LLG_J 3LU0_D 4LK0_J 5TBZ_I 5W1S_J 6WMU_D 4LK0...,ELECTRON MICROSCOPY,X-RAY DIFFRACTION ELECTRON MICROSCOPY,"transcription/dna, transcription, transferase..."
3,161,7NKY_B,1I50_B 4BBR_B 1SFO_B 1NIK_B 1TWA_B 3QT1_B 3RZD...,ELECTRON MICROSCOPY,X-RAY DIFFRACTION ELECTRON MICROSCOPY,"transferase transferase, transcription, trans..."
4,207,2AGP_A,2W9A_A 2W8K_A 3FDS_A 2W9B_B 2RDJ_B 2W9C_B 2W8L...,X-RAY DIFFRACTION,X-RAY DIFFRACTION,"transferase/dna,transferase transferase/dna dn..."
...,...,...,...,...,...,...
139,32559,1ZVV_B 1ZVV_A 1ZVV_G,2JCG_A,X-RAY DIFFRACTION,X-RAY DIFFRACTION,"transcription,transcription/dna"
141,33173,3UDG_A 3UDG_C 3UDG_B,1SE8_A,X-RAY DIFFRACTION,X-RAY DIFFRACTION,"dna binding protein,dna binding protein/dna"
148,34877,7CSZ_A,7CSX_A,X-RAY DIFFRACTION,X-RAY DIFFRACTION,"rna binding protein/dna,rna binding protein"
149,36247,2CCZ_A,1V1Q_A 1V1Q_B,X-RAY DIFFRACTION,X-RAY DIFFRACTION,"dna/replication,dna binding"


# FATCAT

## Download unbound structures

In [46]:
all_unbound = uniq([j[:4] for i in df_bound_unbound.unboundId for j in i.split(" ")])
failed_dl = []

def dl_str(f):
    ####
    if os.path.exists(f"pdb_str/{f}.pdb"):
        return
    ####
    url = f"https://files.rcsb.org/download/{f}.pdb"
    success = download_file(url, f"pdb_str/{f.lower()}.pdb")
    if success == False:
#         print(f)
        failed_dl.append(f)
    
# Run 5 multiple threads. Each call will take the next element in urls list
results = ThreadPool(50).imap_unordered(dl_str, all_unbound)
for r in tqdm(results, total = len(all_unbound)):
    pass

  0%|          | 0/492 [00:00<?, ?it/s]

## Prepare files to compute RMSD : split PDB for chains

In [47]:
df_bound_unbound.boundId

0                                                 1RCN_E
1      6N4C_C 6GH5_C 6XL5_C 6PST_I 3IYD_C 6XH7_C 7C17...
2                     6GH5_D 6XL5_D 3IYD_D 6Z9T_Y 6GFW_D
3                                                 7NKY_B
4                                                 2AGP_A
                             ...                        
139                                 1ZVV_B 1ZVV_A 1ZVV_G
141                                 3UDG_A 3UDG_C 3UDG_B
148                                               7CSZ_A
149                                               2CCZ_A
151                                               5ODL_A
Name: boundId, Length: 105, dtype: object

In [ ]:
def get_line_description(line):
    return line[12:16] + " " + line[17:20] + " " + line[22:26]

failed_split = []
    
all_unbound = uniq([j for i in df_bound_unbound.unboundId for j in i.split(" ")])
all_bound = uniq([j for i in df_bound_unbound.boundId for j in i.split(" ")])
# Save only the chains
total = len(all_unbound)
for s in tqdm(all_bound + all_unbound):
    sl = s.lower()
    chain_name = f"./pdb_chains/{sl}.pdb"
    
    if os.path.exists(chain_name):
        continue
    try:
        mol = molecule.load("pdb", f"pdb_str/{sl[:4]}.pdb")
    except Exception as e:
#         print(e)
        failed_split.append(sl[:4])
        continue
    sel = atomsel(f"noh and chain '{s[5:]}'", mol)
    sel.write("pdb", chain_name)
    molecule.delete(mol)

    #Delete duplicate lines (~Altloc)
    pdb_content = open(chain_name, "r").read().split("\n")
    for line in range(len(pdb_content)-1, 0, -1):
        if get_line_description(pdb_content[line]) == get_line_description(pdb_content[line-1]):
            del(pdb_content[line])
    open(chain_name, "w").write("\n".join(pdb_content))
    
    #Add missing atoms, remove water, ...
    try:
        fix = pdbfixer.PDBFixer(filename=chain_name)
        fix.findMissingResidues()
        fix.findMissingAtoms()
        fix.addMissingAtoms()
        fix.removeHeterogens(keepWater=False)
        fix.topology.createStandardBonds()
        PDBFile.writeFile(fix.topology, fix.positions, open(chain_name, 'w'), keepIds = True)
    except:
        print("PDB fix err.", chain_name)
        continue

  0%|          | 0/1327 [00:00<?, ?it/s]

In [ ]:
failed_split = uniq(failed_split)
failed_split

### Interlude : remove failed split from tables 

In [ ]:
def remove_failed(cell, failed_split):
    for s in cell.split(" "):
        if s[:4].lower() in failed_split:
            cell = cell.replace(s, "")
            cell = cell.replace("  ", " ")
            if cell[0] == " ":
                cell = cell[1:]
            if cell[-1] == " ":
                cell = cell[:-1]
    return cell

for i, r in df_bound_unbound.iterrows():
    b = r.boundId
    ub = r.unboundId
    

    df_bound_unbound.loc[i].boundId = remove_failed(b, failed_split)
    df_bound_unbound.loc[i].unboundId = remove_failed(ub, failed_split)
    


## Prepare command to compute RMSD using FATCAT

In [ ]:
FATCAT_PATH = "./utils/FATCAT/FATCATMain/FATCAT"
# fatcat_out = open("./fatcat_out.txt", "a")
list_of_commands = ""
count = 0

for r in tqdm(df_bound_unbound.iterrows(), total = len(df_bound_unbound)):
    bound = r[1].boundId.split(" ")
    unbound = r[1].unboundId.split(" ")
    
    all_ch = sorted(bound+unbound)
    for i1,j1 in enumerate(all_ch[:]):
        for i2,j2 in enumerate(all_ch[i1:]):
            if j1[:4] in failed_split or j2[:4] in failed_split:
                continue
            count += 1
            # Do FATCAT
            list_of_commands += f"{FATCAT_PATH} -o results_fatcat/{j1.lower()}_{j2.lower()} -p1 {j1.lower()}.pdb -p2 {j2.lower()}.pdb -i ./pdb_chains -r -q -c\n"

print(count)
open("./fatcat_commands.txt", "w").write(list_of_commands)

df_bound_unbound.to_csv("./csv_tables/bound_unbound.preFATCAT.csv")

## Run FATCAT

# Super-impose structures in cluster to check interface surface and ssDNA position

## Superimpose PDB

In [ ]:
def read_fatcat_res(i1, i2, fatcat_results):
    if f"./results_fatcat/{i1.lower()}_{i2.lower()}.chain.txt" in fatcat_results:
        radix = f"{i1.lower()}_{i2.lower()}"
        inv = False
    elif f"./results_fatcat/{i2.lower()}_{i1.lower()}.chain.txt" in fatcat_results:
        radix = f"{i2.lower()}_{i1.lower()}"
        inv = True
    else:
        return None, None, None, None
    ftr = f"./results_fatcat/{radix}.chain.txt"
            
    with open(ftr, "r") as ft:
        ftc = ft.read().split("\n\n")

    afp = ftc[3]
    afp = afp.split("\n")[1:-1]
    afp_1 = [l.split(" ") for l in afp[1::3]]
    afp_1 = [int(item) for sublist in afp_1 for item in sublist if item != ""]
    afp_2 = [l.split(" ") for l in afp[2::3]]
    afp_2 = [int(item) for sublist in afp_2 for item in sublist if item != ""]

    return afp_1, afp_2, radix, inv

In [ ]:
fatcat_results = set(glob("./results_fatcat/*chain*"))
pdb_chains = glob(f"./pdb_chains/*")
pdb_ssDNA  = glob(f"./pdb_ssDNAProtChains/*")
pdb = pdb_chains + pdb_ssDNA

only_interactions = pd.read_csv("csv_tables/ssDNA_interacting.csv", index_col = 0)
df_bound_unbound = pd.read_csv("./csv_tables/bound_unbound.preFATCAT.csv", index_col = 0)

## Process Fatcat result files

In [ ]:
fatcat_results = set(glob("./results_fatcat/*chain*"))
df_bound_unbound = pd.read_csv("./csv_tables/bound_unbound.preFATCAT.csv", index_col = 0)
dict_fatcat = {}

for i,(cl, b,ub, _,_,_,) in tqdm(df_bound_unbound.iterrows(), total = len(df_bound_unbound)):
    b = b.split(" ")
    ub = ub.split(" ")
    
    for i1 in b+ub:
        afp_1, afp_2, radix, inv = read_fatcat_res(i1, i1, fatcat_results)
        if afp_1 is None or afp_2 is None:
            continue
        if len(afp_1) != len(afp_2):
            print(i1, i2)
        dict_fatcat[radix] = [{r : ind for ind,r in enumerate(afp_1)}, {r : ind for ind, r in enumerate(afp_2)}]
    
    for i1, i2 in combinations(b+ub, 2):
        afp_1, afp_2, radix, inv = read_fatcat_res(i1, i2, fatcat_results)
        if afp_1 is None or afp_2 is None:
            continue
        if len(afp_1) != len(afp_2):
            print(i1, i2)
        
        dict_fatcat[radix] = [{r : ind for ind,r in enumerate(afp_1)}, {r : ind for ind, r in enumerate(afp_2)}]

## Split according to ssDNA binding interfacenetworkx

In [ ]:
def get_data(str_x, only_interactions):
    data = get_name_data(str_x)
    s = only_interactions[(only_interactions["structureId"] == data[0])
                        & (only_interactions["chainId"] == data[1])
                        & (only_interactions["int_ssDNA_chain"] == data[2])]
    
    ssDNA_int = flatten(list(map(eval, s.ssDNA_int)))
    ssDNA_int = "' '".join([f"{i}" for i in ssDNA_int])
    
    return s, ssDNA_int, data[0], data[1], data[2]

def flatten(l):
    r = [item for sublist in l for ssublist in sublist for item in ssublist]
    return r

def get_name_data(str_x):
    data = str_x.split("/")[-1].split("_")
    pdb = data[0].upper()
    pc = data[1]
    dc = data[2][:-4]
    
    return pdb, pc, dc

def def_contacts(str_1, mol_1, only_interactions, th):
    s_1, ssDNA_int_1,pdb_1,pc_1,dc_1 = get_data(str_1, only_interactions)
    q = f"chain {pc_1} and (within {th} of (chain {dc_1} and resid '{ssDNA_int_1}'))"
    try:
        sel_1 = atomsel(q, mol_1)
#         if "6pb5_C_2" in str_1:
#             print(sel_1)
    except:
        print(q, str_1,ssDNA_int_1)
        return None, None, None
#     if len(sel_1) == 0:
#         print(s_1)
    return sel_1, dc_1, s_1

def get_radix(i1, i2, dict_fatcat):
    if f"{i1[0].lower()}_{i2[0].lower()}" in dict_fatcat.keys():
        return f"{i1[0].lower()}_{i2[0].lower()}"
    elif f"{i2[0].lower()}_{i1[0].lower()}" in dict_fatcat.keys():
        return f"{i2[0].lower()}_{i1[0].lower()}"
    else:
        return None

def clean_int(str_1, s_1, only_interactions, df_bound_unbound):
    data = get_name_data(str_1)
    only_interactions.drop(s_1.index[0], inplace = True)
    s = only_interactions[(only_interactions["structureId"] == data[0])
     & (only_interactions["chainId"] == data[1])]
#     print("d", s)
    if len(s) == 0:
        df_bound_unbound.loc[i,"boundId"] = df_bound_unbound.loc[i,"boundId"].replace(f"{data[0]}_{data[1]}", "").strip().replace("  ", " ")

In [ ]:
def find_proteins_interacting_with_ssDNA(ldict_ssDNA, only_interactions, th):
    ldict_contact = {}
    for i1 in ldict_ssDNA.items():
        for str_1, mol_1 in zip(*i1[1]):
            contacts, dc_1, s_1 = def_contacts(str_1, mol_1, only_interactions, th)
            if contacts is None:
                continue
            # If no contact, delete it
            if len(contacts) == 0:
                clean_int(str_1, s_1, only_interactions, df_bound_unbound)
            ldict_contact[i1[0]+"_"+dc_1] = set(contacts.resid)
    return ldict_contact

def get_coms(str_1, str_2, ref_1, ref_2):
    pdb_1,pc_1,dc_1 = get_name_data(str_1)
    sel_1 = ldict_contact[f"{pdb_1.lower()}_{pc_1}_{dc_1}"]
    com_1 = set([ref_1[res] for res in set(sel_1) if res in ref_1.keys()])
        
    pdb_2,pc_2,dc_2 = get_name_data(str_2)
    sel_2 = ldict_contact[f"{pdb_2.lower()}_{pc_2}_{dc_2}"]
    com_2 = set([ref_2[res] for res in set(sel_2) if res in ref_2.keys()])
    return com_1, com_2

def get_sq_pd(lssDNA, fn):
    sq_np = np.ones((len(lssDNA), len(lssDNA)))
    np.fill_diagonal(sq_np, 0)
    return pd.DataFrame(data=sq_np, index=fn, columns=fn)

In [ ]:
only_interactions = pd.read_csv("csv_tables/ssDNA_interacting.csv", index_col = 0)
df_bound_unbound = pd.read_csv("./csv_tables/bound_unbound.preFATCAT.csv", index_col = 0)
split_bound_unbound = pd.DataFrame(columns = df_bound_unbound.columns)
new_idx = []

for i,(cl, b,_,_,_,_,) in tqdm(df_bound_unbound.iterrows(), total = len(df_bound_unbound)):
    b = b.split(" ")
    cb = len(b)
#     if i != 2:
#         continue
    if cb == 1: 
        print("u", i)
        split_bound_unbound = split_bound_unbound.append(df_bound_unbound.loc[i])
        new_idx.append(i)
        continue
        
    th = 5.0
    ldict_ssDNA = {f"{j[:4].lower()}_{j[5:].upper()}" : [glob(f"./pdb_ssDNAProtChains/{j[:4].lower()}_{j[5:].upper()}*"), [molecule.load("pdb", k) for k in glob(f"./pdb_ssDNAProtChains/{j[:4].lower()}_{j[5:].upper()}*")]] for j in b}

    # Find protein residues interacting with ssDNA
    ldict_contact = {}
    ldict_contact =find_proteins_interacting_with_ssDNA(ldict_ssDNA, only_interactions, th)
    
    # Second check, after cleaning
    b = df_bound_unbound.loc[i, "boundId"]
    b = b.split(" ")
    cb = len(b)
    if cb == 1: 
        print("u", i)
        split_bound_unbound = split_bound_unbound.append(df_bound_unbound.loc[i])
        new_idx.append(i)
        continue
    
    ldict_ssDNA = {f"{j[:4].lower()}_{j[5:].upper()}" : [glob(f"./pdb_ssDNAProtChains/{j[:4].lower()}_{j[5:].upper()}*"), 
                                                        [molecule.load("pdb", k)
                                                        for k in glob(f"./pdb_ssDNAProtChains/{j[:4].lower()}_{j[5:].upper()}*")]]
                   for j in b}
    lssDNA = [[j,[k, molecule.load("pdb", k)]] for j in b for k in glob(f"./pdb_ssDNAProtChains/{j[:4].lower()}_{j[5:].upper()}*")]
    fn = [k for j in b for k in glob(f"./pdb_ssDNAProtChains/{j[:4].lower()}_{j[5:].upper()}*")]
    comb = list(combinations(lssDNA, 2))[:]
    no_double = True
    double_count = 0
    
    sq_pd = get_sq_pd(lssDNA, fn)
    
    for i1, i2 in comb:
        radix = get_radix(i1, i2, dict_fatcat)
#         print(radix)
        if radix is None:
            continue
        ref_1, ref_2 = dict_fatcat[radix]
        
        # List Bound pairs to compare ssDNA interface
        (str_1, mol_1), (str_2, mol_2) = i1[1], i2[1]
        com_1, com_2 = get_coms(str_1, str_2, ref_1, ref_2)
                
        ui = com_1.intersection(com_2)
        if len(ui) == 0:
            sq_pd.loc[str_1, str_2] = 0
            sq_pd.loc[str_2, str_1] = 0
            no_double = False
            double_count+=1

#     print(sq_pd.sum())
    if no_double:
        print("s", i)
        split_bound_unbound = split_bound_unbound.append(df_bound_unbound.loc[i])
        new_idx.append(i)
        pass
    else:
        G = nx.from_pandas_adjacency(sq_pd)
        lG = [len(c) for c in sorted(nx.connected_components(G), key=len, reverse=True)]
        if len(lG) == 1:
            split_bound_unbound = split_bound_unbound.append(df_bound_unbound.loc[i])
            new_idx.append(i)
        else:
            for idx, c in enumerate(nx.connected_components(G)):
                split_bound_unbound = split_bound_unbound.append(df_bound_unbound.loc[i])
                new_bId = map(str.upper, c)
                new_bId = [x.split("/")[-1][:-4] for x in new_bId]
                new_bId = " ".join(new_bId)
                split_bound_unbound.iloc[-1, split_bound_unbound.columns.get_loc('boundId')] = new_bId
                new_idx.append(f"{i}.{idx+1}")
        print("d", i, double_count, len(comb), lG)
    
    for mol in molecule.listall():
        molecule.delete(mol)
split_bound_unbound.index = new_idx
for mol in molecule.listall():
    molecule.delete(mol)
#     break

# Remove unbound structure with missing interface residues

In [ ]:
fatcat_results = set(glob("./results_fatcat/*chain*"))
only_interactions = pd.read_csv("csv_tables/ssDNA_interacting.csv", index_col = 0)
pdb_chains = set(glob("./pdb_chains/*"))

In [ ]:
def most_frequent(List):
    return max(set(List), key = List.count)

In [ ]:
nan_value = float("NaN")
interface_pd = split_bound_unbound.copy(deep = True)

for i,(cl, b,ub, _,_,_,) in tqdm(interface_pd.iterrows(), total = len(interface_pd)):
#     if i != 0:
#         continue
    b = b.split(" ")
    ub = ub.split(" ")
    
    th = 5.0
    ldict_ssDNA = {f"{j[:4].lower()}_{j[5:].upper()}" : [glob(f"./pdb_ssDNAProtChains/{j[:4].lower()}_{j[5:].upper()}*"), [molecule.load("pdb", k) for k in glob(f"./pdb_ssDNAProtChains/{j[:4].lower()}_{j[5:].upper()}*")]] for j in b}
    ldict_chain = {j.lower() : [molecule.load("pdb", f"./pdb_chains/{j.lower()}.pdb")] for j in ub if f"./pdb_chains/{j.lower()}.pdb" in pdb_chains}
    for k, mol in ldict_chain.items():
        ldict_chain[k].append(set(atomsel("all", mol[0]).resid))
    
    lssDNA = [[j,[k, molecule.load("pdb", k)]] for j in b for k in glob(f"./pdb_ssDNAProtChains/{j[:4].lower()}_{j[5:].upper()}*")]
    ldict_contact =find_proteins_interacting_with_ssDNA(ldict_ssDNA, only_interactions, th)
    
    ub_ok = []
    for bcont, (bf, mol_1) in lssDNA:
        bcont = "_".join(bcont.split("_")[:2])
        ssCont = set([it for k in ldict_contact.items()  for it in k[1] if bcont.upper() == k[0][:len(bcont)].upper()])
#         print(ssCont)
        partial_ub_ok = []
        for ubs in ub:
            afp_1, afp_2, radix, inv = read_fatcat_res(bcont, ubs, fatcat_results)
            if afp_1 is None or afp_2 is None:
                continue
            if not inv:
                afp_b, afp_ub = afp_1, afp_2
            else:
                afp_ub, afp_b = afp_1, afp_2
            
            diff = set([iub - ib for ib, iub in zip(afp_b, afp_ub)])
            if len(diff) == 1:
                ofset = list(diff)[0]
            else:
                ofset = most_frequent([iub - ib  for ib, iub in zip(afp_b, afp_ub)])
#             print(bcont, ubs, ofset)
#             print(afp_1)
#             print(afp_2)
                
            ubCont = set([resid + ofset for resid in ssCont ])
            ubRes = ldict_chain[ubs.lower()][1]
#             print(ubRes)
            diff_bUb = ubCont.difference(ubRes)
#             print(diff_bUb)
            if len(diff_bUb) != 0 or True:
                partial_ub_ok.append(ubs)
                continue
#             else:
#                 print(bcont, ubs, radix, diff)
#                 print(ubRes)
#                 print(ubCont)
        ub_ok.append(set(partial_ub_ok))
    interface_pd.loc[i,"unboundId"] = " ".join(set.intersection(*ub_ok))
                
#             break
#         break
#     break
interface_pd = interface_pd.replace("", nan_value, inplace=False).dropna(subset = ["unboundId"], inplace=False)


In [ ]:
cl = interface_pd[f"clusterNumber{clusterNumber}"].to_list()
idx = interface_pd.index

cur_idx = 0
sub_idx = 1
new_idx = []
for i in range(len(cl)-1):
    if cl[i] != cl[i+1] and sub_idx == 1:
        new_idx.append(str(cur_idx))
        cur_idx += 1
    if cl[i] != cl[i+1] and sub_idx != 1:
        new_idx.append(f"{cur_idx}.{sub_idx}")
        sub_idx = 1
        cur_idx += 1
    if cl[i] == cl[i+1]:
        new_idx.append(f"{cur_idx}.{sub_idx}")
        sub_idx += 1
if cl[-1] == cl[-2]:
    new_idx.append(f"{cur_idx}.{sub_idx}")
else:
    new_idx.append(str(cur_idx))

interface_pd.index = new_idx

In [ ]:
interface_pd.to_csv("./csv_tables/bound_unbound.cleaned.csv")

In [ ]:
interface_pd

# RMSD

## Convert PDB_id pairs into RMSD matrix

### Read RMSD from fatcat results

In [ ]:
fatcat_fileList = glob("./results_fatcat/*")

In [ ]:
rmsdList = []
for f in tqdm(fatcat_fileList[:]):
    s1,c1,s2,c2 = f[17:-10].split("_")
    try:
        fc = open(f, "r").read().split("\n")
        RMSD_line = fc[15]
        r = float(RMSD_line.split(" ")[7][:-1])
        rmsdList.append([f"{s1}_{c1}", f"{s2}_{c2}", r])
    except Exception as e:
        print(f)
        pass
#     break

### Convert list of RMSD into dict of dict then into dict of Tables/dataframes

In [ ]:
rmsdDict = {}
for l in rmsdList:
    ####
    if l[0] in rmsdDict.keys():
        rmsdDict[l[0]][l[1]] = l[2]
    else:
        rmsdDict[l[0]] = {l[1]:l[2]}
    ####
    if l[1] in rmsdDict.keys():
        rmsdDict[l[1]][l[0]] = l[2]
    else:
        rmsdDict[l[1]] = {l[0]:l[2]}

In [ ]:
rmsdDict['6gh5_d']['5vsw_d']

In [ ]:
failed_rmsd = []
rmsd_tables = {}
# split_bound_unbound = pd.read_csv("csv_tables/bound_unbound.cleaned.csv", index_col = 0)

for i, r in tqdm(interface_pd.iterrows(), total = len(interface_pd)):
    bounds = r.boundId.split(" ")
    bounds = ["_".join(b.split("_")[:2]) for b in bounds]
    unbounds = r.unboundId.split(" ")
    str_names = [""+b.lower()+".pdb" for b in bounds] + [""+ b.lower() +".pdb" for b in unbounds]
    str_names_forMatrix = ["(b)"+b.lower() for b in bounds] + ["(ub)"+ b.lower() for b in unbounds]
    
    d = pd.DataFrame(0, index=str_names_forMatrix, columns=str_names_forMatrix)
    
    for i1, (m1, n1) in enumerate(zip(str_names, str_names_forMatrix)):
        tmp_col = []
        for i2, (m2, n2) in enumerate(zip(str_names, str_names_forMatrix)):
            if i2 == i1 :
                tmp_col.append(0)
                continue
            try:
                tmp_col.append(rmsdDict[m1[:-4]][m2[:-4]])
            except KeyError:
                tmp_col.append(0.0)
                if m1 not in rmsdDict.keys():
                    failed_rmsd.append(m1)
                if m2 not in rmsdDict.keys():
                    failed_rmsd.append(m2)
        d.loc[n1, :] = tmp_col
    
    rmsd_tables[i] = d

### TODO : remove failed FATCAT

In [ ]:
len(uniq(failed_rmsd))

# Save all tables

## "Redundant" table, ie with chains ~~from the same main structures~~ with low RMSD (<0.2)

In [ ]:
interface_pd.to_csv("./csv_tables/bound_unbound.redundant.csv", float_format='%.1f')
interface_pd.to_excel("./csv_tables/bound_unbound.redundant.xlsx", float_format='%.1f')

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('./csv_tables/final_sup_RMSD.redundant.xlsx')

# Write each dataframe to a different worksheet.
for i,j in tqdm(zip(rmsd_tables.keys(), rmsd_tables.values()), total=len(rmsd_tables)):
    try:
#         print(j)
        j.to_excel(writer, sheet_name=f'{i}')
    except:
        pass

# Close the Pandas Excel writer and output the Excel file.
writer.save()

## "NON-Redundant" table, ie without UNBOUND chains ~~from the same main structures~~ with low RMSD (<0.2)

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('./csv_tables/final_sup_RMSD.non_redundant.xlsx')
drop_rmsd_tables = rmsd_tables.copy()

def keep_index(close_index, index):
#     print("-", close_index)
#     print("-", index)
    index_to_notkeep = []
    index_to_keep = []
    for i1,i2 in zip(*close_index):
        if i1 < i2:
            index_to_notkeep.append(i2)
        elif i2 not in index_to_notkeep:
            index_to_keep.append(i2)
            
    index_to_keep = uniq(index_to_keep)
    index_to_notkeep = uniq(index_to_notkeep)
#     index_to_notkeep = sorted(list(set(index_to_notkeep).intersection(set(index_to_keep))))
#     name_to_notkeep = [index[tnk] for tnk in index_to_notkeep  if "(ub)" in index[tnk]]
    name_to_notkeep = [index[tnk] for tnk in index_to_notkeep]
    
    return name_to_notkeep

# Write each dataframe to a different worksheet.
# todo = [0, 109]
for i,j in tqdm(zip(drop_rmsd_tables.keys(), drop_rmsd_tables.values()), total=len(drop_rmsd_tables)):
#     if i not in todo:
#         continue
    
    b_index_to_keep = []
    b_index_to_notkeep = []
    ub_index_to_keep = []
    ub_index_to_notkeep = []
    
    index = j.index
    table = j.to_numpy(dtype = float)
    
    bound_names = [k for k in index if "(b)" in k]
    unbound_names = [k for k in index if "(ub)" in k]
    
    bound_table = j.loc[bound_names, bound_names].to_numpy(dtype = float)
    unbound_table = j.loc[unbound_names, unbound_names].to_numpy(dtype = float)
    bound_index = j.loc[bound_names, bound_names].index
    unbound_index = j.loc[unbound_names, unbound_names].index
    
    # Check values under threshold, and exclude diagonal
    close_b = bound_table < 0.2
    close_ub = unbound_table < 0.2
    np.fill_diagonal(close_b, False)
    np.fill_diagonal(close_ub, False)
    
    close_b_index = np.where(close_b == True)
    close_ub_index = np.where(close_ub == True)
        
    # Get one name to keep, remove the others
    b_name_to_notkeep  = keep_index(close_b_index, bound_index)
    ub_name_to_notkeep = keep_index(close_ub_index, unbound_index)
    
#     index_to_keep = uniq(index_to_keep)
#     index_to_notkeep = uniq(index_to_notkeep)
#     index_to_notkeep = sorted(list(set(index_to_notkeep).intersection(set(index_to_keep))))
#     name_to_notkeep = [index[tnk] for tnk in index_to_notkeep  if "(ub)" in index[tnk]]
        
    # Modify table
    drop_rmsd_tables[i] = drop_rmsd_tables[i].drop(b_name_to_notkeep)
    drop_rmsd_tables[i] = drop_rmsd_tables[i].drop(b_name_to_notkeep, axis = 1)
    drop_rmsd_tables[i] = drop_rmsd_tables[i].drop(ub_name_to_notkeep)
    drop_rmsd_tables[i] = drop_rmsd_tables[i].drop(ub_name_to_notkeep, axis = 1)
    
#     if i in todo:
#         print(table)
#         print(drop_rmsd_tables[i])
#         print(b_name_to_notkeep)
    
    try:
        drop_rmsd_tables[i].to_excel(writer, sheet_name=f'{i}')
    except:
        pass

# Close the Pandas Excel writer and output the Excel file.
writer.save()

### Update main table

In [ ]:
unboundId = []
boundId = []

for i,j in tqdm(zip(drop_rmsd_tables.keys(), drop_rmsd_tables.values()), total=len(drop_rmsd_tables)):
    # Get unbound names from the modified RMSD table
    j_ub = [n for n in j.index if "(ub)" in n]
    j_ub = [n.replace("(ub)", "") for n in j_ub]
    j_ub = [n.upper() for n in j_ub]
    ub_list = " ".join(j_ub)
    
    # Modify the table unbound value
    unboundId.append(ub_list)
    
    ####    ####    ####    ####    ####    ####    ##
    # Get unbound names from the modified RMSD table #
    ####    ####    ####    ####    ####    ####    ##
    j_b = [n for n in j.index if "(b)" in n]
    j_b = [n.replace("(b)", "") for n in j_b]
    j_b = [n.upper() for n in j_b]
    b_list = " ".join(j_b)
    
    # Modify the table unbound value
    boundId.append(b_list)
    
interface_pd.unboundId = unboundId
interface_pd.boundId = boundId

# df_new_index.replace('', np.nan, inplace=True)
# df_new_index.dropna(inplace=True)
    
interface_pd.to_csv("./csv_tables/bound_unbound.non_redundant.csv", float_format='%.1f')
interface_pd.to_excel("./csv_tables/bound_unbound.non_redundant.xlsx", float_format='%.1f')

In [ ]:
interface_pd

# Sequence homology table

In [ ]:
cluster_src = "https://cdn.rcsb.org/resources/sequence/clusters/"
#"bc-30.out"
cluster_pct = [100, 95, 90, 70, 50, 40, 30]

In [ ]:
for pct in tqdm(cluster_pct):
    download_file(f"{cluster_src}bc-{pct}.out", f"rcsb_clustering/bc-{pct}.out")

In [ ]:
pct_to_member = {}
for pct in cluster_pct:
    clfile = open(f"rcsb_clustering/bc-{pct}.out", "r").read().split("\n")
    
    pct_to_member[pct] = [content.split(" ") for content in clfile if content != ""]

In [ ]:
pct_to_member_to_cl = {}
for pct in tqdm(cluster_pct):
    pct_to_member_to_cl[pct] = {}
    members_list = pct_to_member[pct]
    for i, members in enumerate(members_list):
        for member in members:
            pct_to_member_to_cl[pct][member] = i

In [ ]:
first_bound_list = []
for i, (cl, b, ub, _, _, _,) in tqdm(interface_pd.iterrows(), total = len(interface_pd)):
    b = b.split(" ")[0]
    b = "_".join(b.split("_")[:2])
    first_bound_list.append((i, b))

In [ ]:
sq_np = np.zeros([len(interface_pd), len(interface_pd)])
np.fill_diagonal(sq_np, 100)
sq_pd = pd.DataFrame(data=sq_np, index=interface_pd.index, columns=interface_pd.index)

In [ ]:
comb = permutations(first_bound_list, 2)
for perm in list(comb):
    for pct in cluster_pct:
        try:
            cl_1 = pct_to_member_to_cl[pct][perm[0][1]]
            cl_2 = pct_to_member_to_cl[pct][perm[1][1]]
            if cl_1 == cl_2:
                sq_pd.loc[perm[0][0], perm[1][0]] = pct
                break
        except:
            sq_pd.loc[perm[0][0], perm[1][0]] = "e"
            continue
sq_pd = sq_pd.applymap(lambda x: int(x) if x != "e" else x)

In [ ]:
sq_pd.to_csv("csv_tables/final_sup_clusterHomology.csv", sep = "\t")